<a href="https://colab.research.google.com/github/giangkarry/Machine-Learning/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/giangkarry/Machine-Learning.git

fatal: destination path 'Machine-Learning' already exists and is not an empty directory.


In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
classes = ('2C', '3C', '4C')
TrainTest = namedtuple('TrainTest', ['train', 'test'])
traindir = '/content/Machine-Learning/DATA_CHAMBER_2021/train'
testdir = '/content/Machine-Learning/DATA_CHAMBER_2021/test'
mean_, std_ = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

#raw_img
def raw_img(img_size):
  transform_train = transforms.Compose([
        transforms.Resize((img_size,img_size)), 
        transforms.ToTensor()
    ])
  transform_test = transforms.Compose([
        transforms.Resize((img_size,img_size)), 
        transforms.ToTensor()
    ])
  return transform_train, transform_test

# preprocess_image
def preprocess_img(img_size):
  transform_train = transforms.Compose([
       transforms.Resize((img_size, img_size)),
       transforms.GaussianBlur(3),
       transforms.RandomRotation(2.8),
       transforms.ToTensor(),
       #transforms.Normalize(mean=mean_, std=std_)                                 
                                        
  ])

  transform_test = transforms.Compose([
        transforms.Resize((img_size,img_size)), 
        transforms.ToTensor()
    ])
  return transform_train, transform_test

# augmentation_image
def augmentation_img(img_size):
  transform_train = transforms.Compose([
       transforms.Resize((img_size, img_size)), 
       transforms.RandomCrop(img_size, padding = 4),
       transforms.RandomHorizontalFlip(),
       transforms.ColorJitter(brightness=0.3, contrast=0.1),
       transforms.ToTensor()                               
  ])

  transform_test = transforms.Compose([
        transforms.Resize((img_size,img_size)), 
        transforms.ToTensor()
    ])
  return transform_train, transform_test

In [ ]:
#hàm chuẩn bị dữ liệu
def prepare_data(img_size):
  #transform_train, transform_test = raw_img(img_size)
  #transform_train, transform_test = preprocess_img(img_size)
  transform_train, transform_test = augmentation_img(img_size)
  
  trainset = torchvision.datasets.ImageFolder(root= traindir, transform=transform_train)
  testset  = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test)
  return TrainTest (
      train = trainset,
      test = testset
  )

#hàm chuẩn bị dữ liệu theo batch đưa vào model
def prepare_loader(datasets):
    batch_size = 32
    num_workers = 4
    trainloader = torch.utils.data.DataLoader(
        dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(
        dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return TrainTest(
        train=trainloader,
        test=testloader
    )

In [ ]:
#train
def get_trainer(model):
  loss = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  return loss, optimizer

#hàm train trong mỗi epoch
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
    model.train()
    running_loss = 0.0
    reporting_step = 42
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % reporting_step == reporting_step-1:
            print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
            running_loss = 0.0

In [ ]:
#hàm test
def test_epoch(epoch, model, loader, device):
    model.eval()
    ypred = []
    ytrue = []
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ypred += list(predicted.cpu().numpy())
      ytrue += list(labels.cpu().numpy())
    return ypred, ytrue

In [ ]:
#thực thi
def main(model = None, img_size = None):
  datasets = prepare_data(img_size)
  loaders = prepare_loader(datasets)
  #print("Tập train: ", len(datasets.train))
  #print("Tập test: ", len(datasets.test))
  #print("class: ", datasets.test.class_to_idx)
  
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  PATH = ''
  if model == 'vgg16':
    model = torchvision.models.vgg16()
    model.classifier[-1] = torch.nn.modules.Linear(in_features=4096, out_features=3)
    PATH = './vgg16.pth'
  
  elif (model == 'vgg19'):
    model = torchvision.models.vgg19()
    model.classifier[-1] = torch.nn.modules.Linear(in_features=4096, out_features=3)
    PATH = './vgg19.pth'
  
  elif model == 'resnet50':
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.Linear(in_features=2048, out_features=3)
    PATH = './vgg16.pth'
  

  n_epoch = 8
  model.to(device)
  #print(device)
  loss, optimizer = get_trainer(model)
  for epoch in range(n_epoch):
    train_epoch(epoch, model, loaders.train, loss, optimizer, device)
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))
    
    
    torch.save(model.state_dict(), PATH)
  return model

In [ ]:
#VGG16
model=main('vgg16', 32)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 Step 41 ave_loss 1.0996
Epoch 0 Step 83 ave_loss 1.0965
Epoch 0 Step 125 ave_loss 1.0954


In [ ]:
#VGG16
model=main('vgg16', 224)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


KeyboardInterrupt: ignored

In [ ]:
#VGG19
model=main('vgg19', 32)

In [ ]:
#VGG19
model=main('vgg19', 224)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
#Reanet50
model = main('resnet50', 32)

In [ ]:
#Reanet50
model = main('resnet50', 224)